# Setup

In [1]:
import json
import requests
from datetime import datetime as dt
from collections import defaultdict
import pandas as pd
import copy

In [2]:
#https://pypi.org/project/gamma-viewer/
from gamma_viewer import GammaViewer
from IPython.display import display, Markdown

In [3]:
def printjson(j):
    print(json.dumps(j,indent=4))
def print_json(j):
    printjson(j)

In [4]:
def post(name,url,message,params=None):
    if params is None:
        response = requests.post(url,json=message)
    else:
        response = requests.post(url,json=message,params=params)
    if not response.status_code == 200:
        print(name, 'error:',response.status_code)
        print(response.json())
        return {}
    return response.json()

def automat(db,message):
    automat_url = f'https://automat.renci.org/{db}/query'
    response = requests.post(automat_url,json=message)
    print(response.status_code)
    return response.json()

def strider(message):
    url = 'https://strider.renci.org/1.1/query'
    strider_answer = post(strider,url,message)
    return strider_answer

def aragorn(message, coalesce_type='xnone'):
    if coalesce_type == 'xnone':
        answer = post('aragorn','https://aragorn.renci.org/1.1/query',message)
    else:
        answer = post('aragorn','https://aragorn.renci.org/1.1/query',message, params={'answer_coalesce_type':coalesce_type})
    return answer

##

def icees(message):
    url = 'https://icees.renci.org:16339/query'
    return post(icees,url,message)

def bte(message):
    url = 'https://api.bte.ncats.io/v1/query'
    return post(strider,url,message)

def coalesce(message,method='all'):
    url = 'https://answercoalesce.renci.org/coalesce/graph'
    return post('AC'+method,url,message)

def striderandfriends(message):
    strider_answer = strider(message)    
    coalesced_answer = post('coalesce','https://answercoalesce.renci.org/coalesce/all',strider_answer)
    omni_answer = post('omnicorp','https://aragorn-ranker.renci.org/omnicorp_overlay',coalesced_answer)
    weighted_answer = post('weight','https://aragorn-ranker.renci.org/weight_correctness',omni_answer)
    scored_answer = post('score','https://aragorn-ranker.renci.org/score',weighted_answer)
    return strider_answer,coalesced_answer,omni_answer,weighted_answer,scored_answer

In [5]:
def print_errors(strider_result):
    errorcounts = defaultdict(int)
    for logmessage in strider_result['logs']:
        if logmessage['level'] == 'ERROR':
            #jm = json.loads(logmessage['message'])
            #words = jm['error'].split()
            #e = " ".join(words[:-5])            
            e = logmessage['error']
            errorcounts[e] += 1
    for error,count in errorcounts.items():
        print(f'{error} ({count} times)')
        
def print_queried_sources(strider_result):
    querycounts = defaultdict(int)
    for logmessage in strider_result['logs']:
        if 'step' in logmessage and isinstance(logmessage['step'],list):
            for s in logmessage['step']:
                querycounts[s['url']] += 1
    for url,count in querycounts.items():
        print(f'{url} ({count} times)')
        
def print_query_for_source(strider_result,url):
    for logmessage in strider_result['logs']:
        if 'step' in logmessage and isinstance(logmessage['step'],list):
            for s in logmessage['step']:
                if s['url']==url:
                    print(s)

In [6]:
def retrieve_ars_results(mid):
    ars='https://ars-dev.transltr.io'
    message_url = f'{ars}/ars/api/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    results = {}
    for child in j['children']:
        if child['actor']['agent'] in ['ara-aragorn', 'ara-aragorn-exp']:
            childmessage_id = child['message']
            child_url = f'{ars}/ars/api/messages/{childmessage_id}'
            child_response = requests.get(child_url).json()
            try:
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except:
                nresults=0
            print( child['status'], child['actor']['agent'],nresults )
    return results

In [7]:
def get_provenance(message):
    """Given a message with results, find the source of the edges"""
    prov = defaultdict(lambda: defaultdict(int)) # {qedge->{source->count}}
    results = message['message']['results']
    kg = message['message']['knowledge_graph']['edges']
    edge_bindings = [ r['edge_bindings'] for r in results ]
    for bindings in edge_bindings:
        for qg_e, kg_l in bindings.items():
            for kg_e in kg_l:
                for att in kg[kg_e['id']]['attributes']:
                    if att['attribute_type_id'] == 'MetaInformation:Provenance':
                        source = att['value']
                        prov[qg_e][source]+=1
    qg_edges = []
    sources = []
    counts = []
    for qg_e in prov:
        for source in prov[qg_e]:
            qg_edges.append(qg_e)
            sources.append(source)
            counts.append(prov[qg_e][source])
    prov_table = pd.DataFrame({"QG Edge":qg_edges, "Source":sources, "Count":counts})
    return prov_table

In [8]:
def ac_to_table(aragorn_result,mnode):
    #scores = []
    answer_node_count = []
    merged_count = []
    method = []
    extra = []
    for res_i, result in enumerate(aragorn_result['message']['results']):
        #scores.append(result['score'])
        answer_node_count.append(len(result['node_bindings']))
        merged_count.append(len(result['node_bindings'][mnode]))
        try:
            method.append(result['node_bindings'][mnode][0]['coalescence_method'])
        except:
            method.append('Original')
    df = pd.DataFrame({'N_Answer_Nodes':answer_node_count, 'N_Merged_Nodes':merged_count, 'Method':method})
    return df

def filter_to_simple(aragorn_result,mnode):
    simple_result = copy.deepcopy(aragorn_result)
    simple_result['message']['results'] = list(
    filter( lambda x: 'coalescence_method' not in x['node_bindings'][mnode][0], 
           aragorn_result['message']['results'])
    )
    print(len(simple_result['message']['results']))
    return simple_result

def print_nodenames(simple_result,qnode):
    #Print the names of the answers
    for result in simple_result['message']['results']:
        #Each answer has an identifier:
        n1_id = result['node_bindings'][qnode][0]['id']
        #The information for that identifier is in the KG:
        node = simple_result['message']['knowledge_graph']['nodes'][n1_id]
        #Each node has a name
        print(node['name'])

def filter_to_coal(aragorn_result,mnode,method):
    #The results that have been coalesced:
    coalesced_results = list(
        filter( lambda x: 'coalescence_method'  in x['node_bindings'][mnode][0], 
               aragorn_result['message']['results'])
    )
    #Those that have been coalesced via a new node (graph coalescence)
    graph_coalesced_results = list(
        filter( lambda x: x['node_bindings'][mnode][0]['coalescence_method'] == method, coalesced_results)
    )
    print(len(graph_coalesced_results))
    simple_result = copy.deepcopy(aragorn_result)
    simple_result['message']['results'] = graph_coalesced_results
    return simple_result

def filter_to_gc(aragorn_result,mnode):
    return filter_to_coal(aragorn_result,mnode,'graph_enrichment')

def filter_to_pc(aragorn_result,mnode):
    return filter_to_coal(aragorn_result,mnode,'property_enrichment')

def print_gc_result(graph,gc_result,node):
    print('p_value:', gc_result['node_bindings'][node][0]['p_value'])
    maxprint=5
    for extra_edge in gc_result['edge_bindings']:
        if not extra_edge.startswith('extra_'):
            continue
        numnodes = len(gc_result['edge_bindings'][extra_edge])
        if numnodes == 0:
            printjson(gc_result)
            return
        print ('Merged',numnodes)
        nprint = min([numnodes,maxprint])
        for eb in gc_result['edge_bindings'][extra_edge][:maxprint]:        
            kge = graph['edges'][eb['id']]
            subject_node = kge['subject']
            object_node = kge['object']
            pred = kge['predicate']
            print( f"  {graph['nodes'][subject_node]['name']} -[{pred}]-> {graph['nodes'][object_node]['name']}")
    print('----')
    
def print_pc_result(knowledge_graph,pc_result,node):
    print('p_value:', pc_result['node_bindings'][node][0]['p_values'])
    print('properties:', pc_result['node_bindings'][node][0]['properties'])
    numnodes = len(pc_result['node_bindings'][node])
    print('node count',numnodes)
    maxprint=5
    nprint = min([numnodes,maxprint])
    for node in pc_result['node_bindings'][node][:nprint]:
        kgn = knowledge_graph['nodes'][node['id']]
        print( f"  {kgn['name']}")
    if numnodes > maxprint:
        print('  ...')
    print('----')

## Query Specific

In [9]:
query={
      "message": {
        "query_graph": {
            "nodes": {
                "n0": {
                    "name": "drug-induced liver injury",
                    "ids": [
                        "SNOMEDCT:197358007"
                    ]
                },
                "n1": {
                    "categories": [
                        "biolink:DiseaseOrPhenotypicFeature"
                    ],
                    "name": "Disease Or Phenotypic Feature"
                },
            },
            "edges": {
                "e0": {
                    "subject": "n0",
                    "object": "n1",
                    "predicates": [
                        "biolink:correlated_with"
                    ]
                }
            }
        }
    }
}


## Strider Direct

In [29]:
#query['log_level']='DEBUG'
start = dt.now()
strider_result = strider(query)
end = dt.now()
print(f"Strider produced {len(strider_result['message']['results'])} results in {end-start}.")

Strider produced 880 results in 0:10:27.719225.


In [31]:
cohdresult = post('cohd','https://cohd.io/api//query',query)

In [33]:
len(cohdresult['message']['results'])

460

In [38]:
n1=[x['node_bindings']['n1'][0]['id'] for x in strider_result['message']['results']]

In [39]:
len(n1)

880

In [40]:
len(set(n1))

440

In [41]:
cohdn1=[x['node_bindings']['n1'][0]['id'] for x in cohdresult['message']['results']]

In [42]:
len(cohdn1), len(set(cohdn1))

(460, 440)

In [44]:
from collections import defaultdict
n1c = defaultdict(int)
for n1 in cohdn1:
    n1c[n1] += 1
for n1,c in n1c.items():
    if c > 1:
        print(n1)

MONDO:0001531
MONDO:0004805
MONDO:0002492
UMLS:C1298758
MONDO:0001106
MONDO:0005570
MONDO:0005240
MONDO:0002280
MONDO:0005281
MONDO:0005137
MONDO:0003406
HP:0002094
HP:0001945
MONDO:0004335
MONDO:0000651
MONDO:0005151
MONDO:0002050


In [45]:
for r in cohdresult['message']['results']:
    if r['node_bindings']['n1'][0]['id'] == 'MONDO:0001531':
        printjson(r)

{
    "edge_bindings": {
        "e0": [
            {
                "id": "ke000019"
            }
        ]
    },
    "node_bindings": {
        "n0": [
            {
                "id": "SNOMEDCT:197358007"
            }
        ],
        "n1": [
            {
                "id": "MONDO:0001531"
            }
        ]
    },
    "score": 2.092464446971226
}
{
    "edge_bindings": {
        "e0": [
            {
                "id": "ke000039"
            }
        ]
    },
    "node_bindings": {
        "n0": [
            {
                "id": "SNOMEDCT:197358007"
            }
        ],
        "n1": [
            {
                "id": "MONDO:0001531"
            }
        ]
    },
    "score": 1.7200262510370588
}
{
    "edge_bindings": {
        "e0": [
            {
                "id": "ke000041"
            }
        ]
    },
    "node_bindings": {
        "n0": [
            {
                "id": "SNOMEDCT:197358007"
            }
        ],
        "n1": [

In [47]:
cohdresult['message']['knowledge_graph']['edges']['ke000019']

{'attributes': [{'attribute_source': 'infores:cohd',
   'attribute_type_id': 'biolink:original_knowledge_source',
   'value': 'infores:cohd',
   'value_type_id': 'biolink:InformationResource',
   'value_url': 'http://cohd.io/api/query'},
  {'attribute_source': 'infores:cohd',
   'attribute_type_id': 'biolink:supporting_data_source',
   'value': 'infores:cohd',
   'value_type_id': 'biolink:InformationResource',
   'value_url': 'http://cohd.io/api/'},
  {'attribute_source': 'infores:cohd',
   'attribute_type_id': 'biolink:p_value',
   'description': 'Chi-square p-value, unadjusted. http://cohd.io/about.html',
   'original_attribute_name': 'p-value',
   'value': 1.5422368522852025e-94,
   'value_type_id': 'EDAM:data_1669',
   'value_url': 'http://edamontology.org/data_1669'},
  {'attribute_source': 'infores:cohd',
   'attribute_type_id': 'biolink:p_value',
   'description': 'Chi-square p-value, Bonferonni adjusted by number of pairs of concepts. http://cohd.io/about.html',
   'original_at

In [48]:
cohdresult['message']['knowledge_graph']['edges']['ke000039']

{'attributes': [{'attribute_source': 'infores:cohd',
   'attribute_type_id': 'biolink:original_knowledge_source',
   'value': 'infores:cohd',
   'value_type_id': 'biolink:InformationResource',
   'value_url': 'http://cohd.io/api/query'},
  {'attribute_source': 'infores:cohd',
   'attribute_type_id': 'biolink:supporting_data_source',
   'value': 'infores:cohd',
   'value_type_id': 'biolink:InformationResource',
   'value_url': 'http://cohd.io/api/'},
  {'attribute_source': 'infores:cohd',
   'attribute_type_id': 'biolink:p_value',
   'description': 'Chi-square p-value, unadjusted. http://cohd.io/about.html',
   'original_attribute_name': 'p-value',
   'value': 9.771991252825492e-64,
   'value_type_id': 'EDAM:data_1669',
   'value_url': 'http://edamontology.org/data_1669'},
  {'attribute_source': 'infores:cohd',
   'attribute_type_id': 'biolink:p_value',
   'description': 'Chi-square p-value, Bonferonni adjusted by number of pairs of concepts. http://cohd.io/about.html',
   'original_att

### Provenance

In [30]:
prov = get_provenance(strider_result)
display(prov)

,QG Edge,Source,Count
0,e0,https://cohd.io/api//query,880


### Queried sources

In [12]:
print_queried_sources(strider_result)

In [14]:
levs=[ l['level'] for l in strider_result['logs']]

In [15]:
n = defaultdict(int)
for l in levs:
    n[l] += 1
for l,c in n.items():
    print(l,c)

DEBUG 123
WARNING 1129
ERROR 6


### Errors

In [16]:
print_errors(strider_result)

504 Server Error: Gateway Time-out for url: https://cohd.io/api//query
For more information check: https://httpstatuses.com/504 (2 times)
 (4 times)


### Results

In [17]:
view = GammaViewer(props={"data":strider_result})
display(view)

### Strider Assessment

Enter Assessment Here

## ARAGORN 

In [28]:
start = dt.now()
aragorn_result = aragorn(query)
end = dt.now()
if 'results' in aragorn_result['message']:
    print(f"ARAGORN produced {len(aragorn_result['message']['results'])} results in {end-start}.")
else:
    print('Error, no result field')

KeyboardInterrupt: 

In [ ]:
view = GammaViewer(props={"data":aragorn_result})
display(view)

In [ ]:
node='n0' #this might need to change based on the query
property_answers = filter_to_pc(aragorn_result,node)
for pc_result in property_answers['message']['results']:
    print_pc_result(aragorn_result['message']['knowledge_graph'],pc_result,node)

In [ ]:
property_answers['message']['results'][0]

In [ ]:
graph_answers=filter_to_gc(aragorn_result,node)
for gc_result in graph_answers['message']['results']:
    print_gc_result(aragorn_result['message']['knowledge_graph'],gc_result,node)

### ARAGORN Assessment

How did we do?